# Moduel

In [26]:
import dotenv
from openai import OpenAI
dotenv.load_dotenv()
import json
from tqdm import tqdm
import ast
import os
import requests
import pandas as pd
from pymilvus import MilvusClient
from pymilvus import model
import numpy as np
from tqdm import tqdm
import time
from pymilvus import MilvusClient, Collection, connections, DataType, CollectionSchema, FieldSchema

# client = OpenAI()

In [2]:
CLOVASTUDIO_API_KEY = os.environ.get("X-NCP-CLOVASTUDIO-API-KEY")
APIGW_API_KEY = os.environ.get("X-NCP-APIGW-API-KEY")

# Milvus Vector DB 생성

In [48]:
client = MilvusClient()

In [15]:
# 필드 스키마 정의
diary_field = FieldSchema(name="diary_id", dtype=DataType.INT64, is_primary=True, description="Diary ID")
user_field = FieldSchema(name="user_id", dtype=DataType.VARCHAR, max_length=100, description="User ID")
text_field = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=10000, description="Diary Text")
dense_vector_field = FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024, description="Dense Embedding Vector")

In [16]:
# 컬렉션 스키마 생성
schema = CollectionSchema(fields=[diary_field, user_field, text_field, dense_vector_field], description="InsideGram Schema")

In [17]:
# # # 컬렉션 생성
# collection_name = "inside_gram"
# client.create_collection(collection_name=collection_name, schema=schema)

# 임베딩 모델 & 함수 

In [40]:
embedding_headers = {
    'X-NCP-CLOVASTUDIO-API-KEY': CLOVASTUDIO_API_KEY,
    'X-NCP-APIGW-API-KEY': APIGW_API_KEY,
    'X-NCP-CLOVASTUDIO-REQUEST-ID': "4f5f51a0-acf0-484e-84c9-46cf2aa067f3",
    'Content-Type': 'application/json; charset=utf-8',
    }

def get_embedding(content):

    request_data = {
        "text": content
        }
    
    reaction = requests.post("https://clovastudio.apigw.ntruss.com/testapp/v1/api-tools/embedding/v2/4aacadacd0c048e98e14e4870c2f2c93", headers=embedding_headers, json=request_data)
    try:
        result = reaction.json()["result"]["embedding"]
    except:
        result = str(reaction)
        print(result)
    return result


# Data Load

In [41]:
file_path = "/home/InsideGram/data/corrected_diary.jsonl"

diary_list = []
with open(file_path, 'r', encoding='utf-8') as file:
    for n, line in enumerate(file):
        diary_list.extend([i["content"] for i in json.loads(json.loads(line.strip())["choices"][0]["message"]["content"])["diaries"]])

In [44]:
entities = []
# error_list = []
for n, diary in tqdm(enumerate(diary_list)):
    user_id = "84c484e2-6bd7-43bb-9a22-a87247730647"
    embedding_i = get_embedding(diary)
    entity_i = {"diary_id":-1, "user_id":user_id, "text":diary, "dense_vector":embedding_i}
    entities.append(entity_i)
    if n%30 == 0:
        time.sleep(60) 
        


176it [06:16,  2.14s/it]


In [52]:
entities_ = []
for i in entities:
    entities_.append({"diary_id":i["diary_field"], "user_id":i["user_field"], "text":i["text_field"], "dense_vector":i["dense_vector_field"]})
    # break

In [58]:
len(entities_)

176

In [55]:
# res = client.insert(collection_name="inside_gram", data = entities_)

# Index

In [61]:
# ######### Dense Vector ##########

# client.release_collection("inside_gram")
# client.drop_index(collection_name="inside_gram", index_name="dense_vector")

# index_params = MilvusClient.prepare_index_params()

# index_params.add_index(
#     field_name="dense_vector",
#     metric_type="COSINE",
#     index_type="FLAT",
#     index_name="dense_vector"
# )

# client.create_index(
#     collection_name="inside_gram",
#     index_params=index_params
# )

# Vector Search

In [67]:
client.load_collection("inside_gram")

In [78]:
diary_text = "오늘은 엄마랑 영화를 봤다. 간만에 엄마와 함께 한 시간이어서 너무 행복했다"

In [79]:
query_vectors = get_embedding(diary_text)

In [93]:
res = client.search(
    collection_name="inside_gram",  # target collection
    data=[query_vectors],  # query vectors
    limit=100,  # number of returned entities
    output_fields=["text"],  # specifies fields to be returned
    # anns_field="dense_vector"
)

In [94]:
res

data: ["[{'id': -1, 'distance': 0.7062965035438538, 'entity': {'text': '오늘은 집에서 혼자 영화를 보면서 하루를 보냈다. 최근에 일이 많아서 스트레스를 받았는데, 영화를 보는 동안만큼은 모든 걸 잊고 힐링할 수 있었다. 코미디 영화를 보면서 웃음이 끊이질 않았다. 정말 필요했던 시간이었다.'}}]"] , extra_info: {'cost': 0}